In [1]:
import os

In [7]:
import tkinter as tk
from PIL import Image, ImageTk, ImageDraw

def add_red_frame(image, frame_width=10):
    # Calculate the size of the new image with the frame
    new_width = image.width + 2 * frame_width
    new_height = image.height + 2 * frame_width
    
    # Create a new image with a red background
    framed_image = Image.new("RGB", (new_width, new_height), "red")
    
    # Paste the original image onto the center of the new image
    framed_image.paste(image, (frame_width, frame_width))
    
    return framed_image

def draw_img(image_path, duration_ms, x, y, frame_width=10):
    # Load the image using PIL
    image = Image.open(image_path)
    
    # Add a red frame around the image
    framed_image = add_red_frame(image, frame_width)
    
    # Create a Tkinter window
    window = tk.Tk()
    window.attributes("-topmost", True)
    window.title("Image Display")
    
    # Convert the framed image to a format Tkinter can use
    tk_image = ImageTk.PhotoImage(framed_image)
    
    # Set window size and position
    window.geometry(f"{framed_image.width}x{framed_image.height}+{x}+{y}")
    
    # Create a label to hold the image
    label = tk.Label(window, image=tk_image)
    label.image = tk_image  # Keep a reference to the image
    label.pack()
    
    # Define a function to close the window
    def close_window():
        window.destroy()
    
    # Schedule the window to close after duration_ms milliseconds
    window.after(duration_ms, close_window)
    
    # Start the Tkinter main loop
    window.mainloop()

In [8]:
def display_images_with_red_frame(folder_path):
    # Get list of image files in the folder
    image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('png', 'jpg', 'jpeg', 'bmp', 'gif'))]
    
    for image_file in image_files:
        # Load image
        image_path = os.path.join(folder_path, image_file)
        draw_img(image_path, 2000, 0, 0)
        
# Folder containing the images
folder_path = 'pics/'  # Replace with your folder path

# Display all images in the folder with a red frame
display_images_with_red_frame(folder_path)

In [2]:
import win32gui
import win32ui
import win32con
import ctypes
from PIL import Image
import time

# Load the image using PIL
image_path = 'img.png'  # Replace with your image file path
image = Image.open(image_path)
image = image.convert('RGB')  # Ensure the image is in RGB mode
image_width, image_height = image.size
image_data = image.tobytes("raw", "BGRX")  # Convert to BGRX format

# Define the BITMAPINFO structure using ctypes
class BITMAPINFOHEADER(ctypes.Structure):
    _fields_ = [
        ("biSize", ctypes.c_uint32),
        ("biWidth", ctypes.c_int32),
        ("biHeight", ctypes.c_int32),
        ("biPlanes", ctypes.c_uint16),
        ("biBitCount", ctypes.c_uint16),
        ("biCompression", ctypes.c_uint32),
        ("biSizeImage", ctypes.c_uint32),
        ("biXPelsPerMeter", ctypes.c_int32),
        ("biYPelsPerMeter", ctypes.c_int32),
        ("biClrUsed", ctypes.c_uint32),
        ("biClrImportant", ctypes.c_uint32),
    ]

class BITMAPINFO(ctypes.Structure):
    _fields_ = [
        ("bmiHeader", BITMAPINFOHEADER),
        ("bmiColors", ctypes.c_uint32 * 3),
    ]

# Create a DC compatible with the screen
hdc = win32gui.GetDC(0)
compatible_dc = win32ui.CreateDCFromHandle(hdc)

# Create a memory DC compatible with the screen DC
memory_dc = compatible_dc.CreateCompatibleDC()

# Create a bitmap object
bitmap = win32ui.CreateBitmap()
bitmap.CreateCompatibleBitmap(compatible_dc, image_width, image_height)

# Select the bitmap into the memory DC
old_bitmap = memory_dc.SelectObject(bitmap)

# Create a bitmap info header
bmi = BITMAPINFO()
bmi.bmiHeader.biSize = ctypes.sizeof(BITMAPINFOHEADER)
bmi.bmiHeader.biWidth = image_width
bmi.bmiHeader.biHeight = -image_height  # Negative to ensure the bitmap is right-side up
bmi.bmiHeader.biPlanes = 1
bmi.bmiHeader.biBitCount = 32
bmi.bmiHeader.biCompression = win32con.BI_RGB

# Set the bits of the bitmap
ctypes.windll.gdi32.SetDIBitsToDevice(
    memory_dc.GetSafeHdc(),
    0, 0, image_width, image_height,
    0, 0, 0, image_height,
    image_data,
    ctypes.byref(bmi),
    win32con.DIB_RGB_COLORS
)

# Draw the image onto the screen
compatible_dc.BitBlt((100, 100), (image_width, image_height), memory_dc, (0, 0), win32con.SRCCOPY)

time.sleep(5)  # Sleep for 5 seconds to view the image

# Clean up
memory_dc.SelectObject(old_bitmap)
memory_dc.DeleteDC()
compatible_dc.DeleteDC()
win32gui.ReleaseDC(0, hdc)
win32gui.DeleteObject(bitmap.GetHandle())  # Corrected line to delete the bitmap object

In [1]:
import tkinter as tk
from PIL import Image, ImageTk
import time 

# Load the image using PIL
image_path = 'img.png'  # Replace with your image file path
image = Image.open(image_path)

# Create a Tkinter window

window = tk.Tk()
window.title("Overlay")
window.geometry(f"{image.width}x{image.height}+300+300")  # Set window size and position
window.attributes("-topmost", True)  # Make the window always on top
window.overrideredirect(True)  # Remove window decorations (optional)

# Convert the image to a format Tkinter can use
tk_image = ImageTk.PhotoImage(image)

# Create a label to hold the image
label = tk.Label(window, image=tk_image)
label.pack()

# Define a function to close the window
def close_window():
    window.destroy()

# Schedule the window to close after 2000 milliseconds (2 seconds)
window.after(20, close_window)

# Start the Tkinter main loop
window.mainloop()
time.sleep(0.5)
    

In [22]:
import win32gui
import win32api
import time
import numpy as np

def get_pixel_color(x, y):
    hdc = win32gui.GetDC(0)  # Get the device context for the entire screen
    color = win32gui.GetPixel(hdc, x, y)  # Get the color of the pixel at (x, y)
    win32gui.ReleaseDC(0, hdc)  # Release the device context
    
    # Extract the RGB components
    red = color & 0xff
    green = (color >> 8) & 0xff
    blue = (color >> 16) & 0xff
    return red, green, blue

def draw_grid(dc, x_c, y_c, width, height, empty_width, offset):
    for x in range(0, width, empty_width):
        for y in range(0, height, empty_width):
            color = win32api.RGB(*(i-offset for i in get_pixel_color(x_c + x, y_c + y)))
            win32gui.SetPixel(dc, x_c + x, y_c + y, color)

# Define the color
red = win32api.RGB(255, 0, 0)
black = win32api.RGB(0, 255, 0)

# Get the device context for the entire screen
dc = win32gui.GetDC(0)

# Define the top-left corner of the square and its size
x, y = 100, 100  # Top-left corner coordinates
width, height = 100, 100  # Size of the square
size = 5  # Size of the square
offset = 70

try:
    draw_grid(dc, x, y, width, height, size, -offset)
    while True:
        # Draw the square
        draw_grid(dc, x, y, width, height, size, 2*offset)
        time.sleep(0.001)
        # Erase the square by drawing it in black (assuming the background is black)
        draw_grid(dc, x, y, width, height, size, -2*offset)
        time.sleep(0.001)
finally:
    # Release the device context
    win32gui.ReleaseDC(0, dc)

KeyboardInterrupt: 

In [2]:
import tkinter as tk
from PIL import ImageTk

def display_image(image_path):
    window = tk.Tk()
    window.title("Steganography Image")
    window.geometry("800x600")  # Adjust the window size as needed

    img = Image.open(image_path)
    tk_image = ImageTk.PhotoImage(img)

    label = tk.Label(window, image=tk_image)
    label.pack()

    window.mainloop()

# Example usage
display_image('img.png')


NameError: name 'Image' is not defined

In [2]:
import tkinter as tk
import numpy as np
from PIL import Image, PngImagePlugin, ImageTk
import io
import time

# Load the image using PIL
image_path = 'img.png'  # Replace with your image file path

def numpy_to_png_image(array: np.ndarray) -> PngImagePlugin.PngImageFile:
    # Ensure the array is in the correct format
    if array.dtype != np.uint8:
        # Scale array to 0-255 range and convert to uint8
        array = ((array - np.min(array)) / (np.max(array) - np.min(array)) * 255).astype(np.uint8)
    
    # Convert the NumPy array to a PIL Image
    pil_image = Image.fromarray(array)
    
    # Save the PIL Image to a BytesIO object
    byte_io = io.BytesIO()
    pil_image.save(byte_io, format='PNG')
    byte_io.seek(0)  # Go to the start of the BytesIO object
    
    # Load the image from the BytesIO object as a PngImageFile
    png_image = PngImagePlugin.PngImageFile(byte_io)
    
    return png_image

def draw_img(png_image, duration_ms, x, y):
    if not isinstance(png_image, PngImagePlugin.PngImageFile):
        raise TypeError("The image must be a PngImagePlugin.PngImageFile object")

    # Create a Tkinter window
    window = tk.Tk()
    
    # Convert the image to a format Tkinter can use
    tk_image = ImageTk.PhotoImage(png_image)
    
    # Set window size and position
    window.geometry(f"{png_image.width}x{png_image.height}+{x}+{y}")
    
    # Create a label to hold the image
    label = tk.Label(window, image=tk_image)
    label.image = tk_image  # Keep a reference to the image
    label.pack()
    
    # Define a function to close the window
    def close_window():
        window.destroy()
    
    # Schedule the window to close after duration_ms milliseconds
    window.after(duration_ms, close_window)
    
    # Start the Tkinter main loop
    window.mainloop()

def draw_qr(path, time_ms, x, y):
    image = Image.open(path)
    img_arr = np.array(image)
    xs = np.linspace(0, image.width, 10, dtype=int)
    ys = np.linspace(0, image.height, 10, dtype=int)
    img_grid = [img_arr[xs[i]:xs[i+1], ys[j]:ys[j+1]] for i in range(9) for j in range(9)]
    
    # Create a Tkinter window
    for img in img_grid:
        png_image = numpy_to_png_image(img)
        draw_img(png_image, time_ms, x, y)
        time.sleep(0.1)
        
draw_qr('img.png', 2000, 100, 100)

KeyboardInterrupt: 